# Documentation of the test cases in test_aabb.cpp
### Troubleshooting
In case this notebook isn't working, make sure the matplotlib widgets are installed and enabled:
https://stackoverflow.com/a/55848505

In [446]:
%matplotlib widget

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [447]:
def get_bb(bmin, bmax):   
    phi = np.arange(1,10,2)*np.pi/4
    Phi, Theta = np.meshgrid(phi, phi)
    
    x_scale = abs(bmax[0]-bmin[0])
    y_scale = abs(bmax[1]-bmin[1])
    z_scale = abs(bmax[2]-bmin[2])

    x = (np.cos(Phi)*np.sin(Theta)+0.5) * x_scale + bmin[0]
    y = (np.sin(Phi)*np.sin(Theta)+0.5) * y_scale + bmin[1]
    z = (np.cos(Theta)/np.sqrt(2)+0.5)  * z_scale + bmin[2]
    
    return x,y,z

def calc_ray(ray_dist, inter_min, inter_max):
    ray_dir = (inter_max-inter_min)
    ray_dir = ray_dir / np.linalg.norm(ray_dir)
    ray_inv_dir = 1/ray_dir
    
    ray_org = inter_min - (ray_dir * ray_dist)     
    
    t_min = np.linalg.norm(inter_min - ray_org)
    t_max = np.linalg.norm(inter_max - ray_org)
    
    ray_dir_sign = np.sign(ray_dir)
    for i in range(len(ray_dir_sign)):
        if ray_dir_sign[i] >= 0:
            ray_dir_sign[i] = 0
        else:
            ray_dir_sign[i] = 1
    ray_dir_sign = ray_dir_sign.astype(np.int)
    
    return ray_org, ray_dir, ray_inv_dir, ray_dir_sign, t_min, t_max

def plot(bmin, bmax, inter_min, inter_max, ray_dist):
    bmin = np.array(bmin)
    bmax = np.array(bmax)
    inter_min = np.array(inter_min)
    inter_max = np.array(inter_max)
    
    ray_org, ray_dir, ray_inv_dir, ray_dir_sign, t_min, t_max = calc_ray(ray_dist, inter_min, inter_max)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    x, y, z = get_bb(bmin, bmax)
    line = np.linspace(ray_org, ray_org + ray_dir * max_t, 100) 
    
    # initialize with viewing angle towards yz-plane
    ax.view_init(10, 1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    # plot bounding box
    ax.plot_surface(x, y, z, alpha=0.2)
    
    # plot ray origin
    ax.scatter(ray_org[0], ray_org[1], ray_org[2])
    
    # plot intersection points
    ax.scatter(inter_min[0], inter_min[1], inter_min[2])
    ax.scatter(inter_max[0], inter_max[1], inter_max[2])    
    
    # plot ray
    ax.plot(line[:,0], line[:,1], line[:,2])
    
    plt.show()
    
    return ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max

def convert_testcase(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign, dtype = 'double', require = True, name = 'EXAMPLE'):
    template = '''
    SECTION("{20}")
    {{
    // return variables    
    {0} tmin;
    {0} tmax;    
    
    // correct values
    {0} tmin_cor = {18};
    {0} tmax_cor = {19};
    
    // precision
    {0} epsilon = std::numeric_limits<{0}>().lowest();
    
    // parameters
    {0} min_t = {1};
    {0} max_t = {2};
    {0} pos_inf = std::numeric_limits<{0}>().max();
    {0} neg_inf = -1 * std::numeric_limits<{0}>().max();
    Vec3r<{0}> bmin{{ {3}, {4}, {5} }};
    Vec3r<{0}> bmax{{ {6}, {7}, {8} }};
    Vec3r<{0}> ray_org{{ {9}, {10}, {11} }};
    Vec3r<{0}> ray_inv_dir{{ {12}, {13}, {14} }};
    Vec3ui ray_dir_sign{{ {15}, {16}, {17} }};
    '''

    if require:
        template = template + '''
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(is_almost_equal(tmin, tmin_cor, epsilon));
    REQUIRE(is_almost_equal(tmax, tmax_cor, epsilon));
    }}
    '''
    else:
        template = template + '''
    REQUIRE_FALSE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    }}
    '''                

    vars = [bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max, min_t, max_t]
    for v in range(len(vars)):
        if dtype == 'double':
            vars[v] = np.array(vars[v]).astype(np.float64)
        else:
            vars[v] = np.array(vars[v]).astype(np.float32)
        vars[v] = np.array(vars[v]).astype(str)
        
        try:
            for i in range(len(vars[v])):            
                if vars[v][i] == 'inf':
                    vars[v][i] = 'pos_inf'
                elif vars[v][i] == '-inf':
                    vars[v][i] = 'neg_inf'  
        except:
                if vars[v] == 'inf':
                    vars[v] = 'pos_inf'
                elif vars[v] == '-inf':
                    vars[v] = 'neg_inf'  

    [bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign, tmin, tmax, min_t, max_t] = vars
    
    template = template.format(dtype, 
                               min_t, max_t, 
                               bmin[0], bmin[1], bmin[2], 
                               bmax[0], bmax[1], bmax[2],
                               ray_org[0], ray_org[1], ray_org[2],
                               ray_inv_dir[0], ray_inv_dir[1], ray_inv_dir[2],
                               int(float(ray_dir_sign[0])), int(float(ray_dir_sign[1])), int(float(ray_dir_sign[2])),
                               tmin, tmax,
                               name
                              )
    print(template)

In [448]:
# parameters
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [1, 2, 2]
inter_min = [0, 1, 1.5]
inter_max = [0, 2, 1.3]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'float',
                 require = True,
                 name = 'float_example'
                )

convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'double',
                 require = True,
                 name = 'double_example'
                )


<ipython-input-447-0d5e5de6747d>:18: RuntimeWarning: divide by zero encountered in true_divide
  ray_inv_dir = 1/ray_dir


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SECTION("float_example")
    {
    // return variables    
    float tmin;
    float tmax;    
    
    // correct values
    float tmin_cor = 1.5;
    float tmax_cor = 2.519804;
    
    // precision
    float epsilon = std::numeric_limits<float>().lowest();
    
    // parameters
    float min_t = 0.0;
    float max_t = 3.0;
    float pos_inf = std::numeric_limits<float>().max();
    float neg_inf = -1 * std::numeric_limits<float>().max();
    Vec3r<float> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<float> bmax{ 1.0, 2.0, 2.0 };
    Vec3r<float> ray_org{ 0.0, -0.470871, 1.7941742 };
    Vec3r<float> ray_inv_dir{ pos_inf, 1.0198039, -5.0990195 };
    Vec3ui ray_dir_sign{ 0, 0, 1 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(is_almost_equal(tmin, tmin_cor, epsilon));
    REQUIRE(is_almost_equal(tmax, tmax_cor, epsilon));
    }
    

    SECTION("double_example")
    {
    // return variables    
    double tmin;
